## Dataset Creation

In [4]:
%pip install bing-image-downloader

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
from bing_image_downloader import downloader

# Define the categories
categories = ['parrot', 'deer']
image_count = 100

# Download the images
for category in categories:
    downloader.download(category, limit=image_count, output_dir='data', adult_filter_off=True, force_replace=False, timeout=60)

[%] Downloading Images to c:\Users\aashm\OneDrive\Desktop\ES335_gradient_thinkers\A4\data\parrot


[!!]Indexing page: 1

[%] Indexed 95 Images on Page 1.


[%] Downloading Image #1 from http://upload.wikimedia.org/wikipedia/commons/4/47/Male_Eclectus_Parrot.jpg
[%] File Downloaded !

[%] Downloading Image #2 from http://upload.wikimedia.org/wikipedia/commons/b/b6/Eclectus_Parrot_RWD.jpg
[%] File Downloaded !

[%] Downloading Image #3 from https://images.pexels.com/photos/1463295/pexels-photo-1463295.jpeg?cs=srgb&amp;dl=animal-beautiful-bright-1463295.jpg&amp;fm=jpg
[%] File Downloaded !

[%] Downloading Image #4 from http://upload.wikimedia.org/wikipedia/commons/0/07/Congo_African_Grey_Parrot_Bali.jpg
[%] File Downloaded !

[%] Downloading Image #5 from https://www.thesprucepets.com/thmb/jP0CorffApXa-NyoAwQfqG1jvRU=/3865x2576/filters:fill(auto,1)/close-up-of-gold-and-blue-macaw-perching-on-tree-962288862-5b50073e46e0fb0037c23c23.jpg
[%] File Downloaded !

[%] Downloading Image #6 from 

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Paths for the original and new directories
original_dir = 'data'
deer_dir = os.path.join(original_dir, 'deer')
parrot_dir = os.path.join(original_dir, 'parrot')

# Create train and test directories for both classes
train_dir = os.path.join(original_dir, 'train')
test_dir = os.path.join(original_dir, 'test')

deer_train_dir = os.path.join(train_dir, 'deer')
deer_test_dir = os.path.join(test_dir, 'deer')
parrot_train_dir = os.path.join(train_dir, 'parrot')
parrot_test_dir = os.path.join(test_dir, 'parrot')

# Create the new directories if they don't exist
os.makedirs(deer_train_dir, exist_ok=True)
os.makedirs(deer_test_dir, exist_ok=True)
os.makedirs(parrot_train_dir, exist_ok=True)
os.makedirs(parrot_test_dir, exist_ok=True)

# Function to split the data
def split_data(src_dir, train_dir, test_dir, test_size=0.2):
    # Get the list of images in the source directory
    files = os.listdir(src_dir)
    # Split the files into train and test
    train_files, test_files = train_test_split(files, test_size=test_size, random_state=42)

    # Move train images
    for file in train_files:
        shutil.move(os.path.join(src_dir, file), os.path.join(train_dir, file))
    
    # Move test images
    for file in test_files:
        shutil.move(os.path.join(src_dir, file), os.path.join(test_dir, file))

# Split the deer and parrot data
split_data(deer_dir, deer_train_dir, deer_test_dir)
split_data(parrot_dir, parrot_train_dir, parrot_test_dir)

# Remove the old directories (if they are empty)
if not os.listdir(deer_dir):
    os.rmdir(deer_dir)  # Remove the empty 'deer' directory

if not os.listdir(parrot_dir):
    os.rmdir(parrot_dir)  # Remove the empty 'parrot' directory

## Binary Classifcation

In [1]:
%pip install tensorflow

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, VGG19
from tensorflow.keras.optimizers import Adam
import os
import time
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import TensorBoard

In [3]:
# Set directory paths
train_dir = 'data/train'
test_dir = 'data/test'

# ImageDataGenerator with Data Augmentation (for the augmented models)
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Rescale pixel values to [0, 1]
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),  # Resize images to 150x150
    batch_size=32,
    class_mode='binary'  # Binary classification (deer vs parrot)
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

Found 164 images belonging to 2 classes.
Found 41 images belonging to 2 classes.


In [4]:
def evaluate_model(model, history):
    # Evaluate on test data
    test_loss, test_accuracy = model.evaluate(test_generator, verbose=0)
    
    # Training metrics
    training_loss = history.history['loss'][-1]
    training_accuracy = history.history['accuracy'][-1]
    
    # Log the results
    return {
        "training_loss": training_loss,
        "training_accuracy": training_accuracy,
        "test_accuracy": test_accuracy,
        "num_parameters": model.count_params()
    }

### VGG (1 Block)

In [11]:
def vgg_1_block():
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
        layers.MaxPooling2D(2, 2),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D(2, 2),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

model_1 = vgg_1_block()
history_1 = model_1.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator,
    verbose=1
)

c:\Users\aashm\OneDrive\Desktop\ES335_gradient_thinkers\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\aashm\OneDrive\Desktop\ES335_gradient_thinkers\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 22s 4s/step - accuracy: 0.3924 - loss: 1.1586 - val_accuracy: 0.6341 - val_loss: 0.6388
Epoch 2/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 18s 3s/step - accuracy: 0.6061 - loss: 0.6738 - val_accuracy: 0.5122 - val_loss: 0.6269
Epoch 3/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 19s 3s/step - accuracy: 0.5612 - loss: 0.5666 - val_accuracy: 0.5854 - val_loss: 0.5544
Epoch 4/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - accuracy: 0.6135 - loss: 0.5457 - val_accuracy: 0.5854 - val_loss: 0.5132
Epoch 5/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - accuracy: 0.6352 - loss: 0.5321 - val_accuracy: 0.8049 - val_loss: 0.4397
Epoch 6/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - accuracy: 0.7550 - loss: 0.4370 - val_accuracy: 0.7561 - val_loss: 0.3947
Epoch 7/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.8161 - loss: 0.4200 - val_accuracy: 0.8293 - val_loss: 0.3316
Epoch 8/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.8250 - loss: 0.4419 - val_accuracy: 0.8537 - val_loss: 0.3200
Epoch 9/10


In [14]:
results_1 = evaluate_model(model_1, history_1)
print(f"Model 1 Results: {results_1}")

Model 1 Results: {'training_loss': 0.3440631330013275, 'training_accuracy': 0.8475610017776489, 'test_accuracy': 0.7317073345184326, 'num_parameters': 10636481}


### VGG (3 blocks)

In [16]:
def vgg_3_blocks():
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
        layers.MaxPooling2D(2, 2),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D(2, 2),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D(2, 2),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

model_2 = vgg_3_blocks()
history_2 = model_2.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator,
    verbose=1
)

c:\Users\aashm\OneDrive\Desktop\ES335_gradient_thinkers\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - accuracy: 0.5447 - loss: 1.5563 - val_accuracy: 0.4878 - val_loss: 0.7038
Epoch 2/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - accuracy: 0.4792 - loss: 0.6984 - val_accuracy: 0.5122 - val_loss: 0.6806
Epoch 3/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.5265 - loss: 0.6833 - val_accuracy: 0.5122 - val_loss: 0.6582
Epoch 4/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - accuracy: 0.4990 - loss: 0.6353 - val_accuracy: 0.5122 - val_loss: 0.7796
Epoch 5/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 20s 3s/step - accuracy: 0.6301 - loss: 0.6038 - val_accuracy: 0.7317 - val_loss: 0.5709
Epoch 6/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 9s 2s/step - accuracy: 0.7682 - loss: 0.5487 - val_accuracy: 0.5610 - val_loss: 0.7343
Epoch 7/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.8291 - loss: 0.4614 - val_accuracy: 0.7073 - val_loss: 0.5384
Epoch 8/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 9s 2s/step - accuracy: 0.7063 - loss: 0.5455 - val_accuracy: 0.7317 - val_loss: 0.5126
Epoch 9/10
6

In [17]:
results_2 = evaluate_model(model_2, history_2)
print(f"Model 2 Results: {results_2}")

Model 2 Results: {'training_loss': 0.4852249324321747, 'training_accuracy': 0.7560975551605225, 'test_accuracy': 0.7804877758026123, 'num_parameters': 4828481}


### VGG (3 blocks) with data augmentation

In [18]:
def vgg_3_blocks_with_augmentation():
    # Set up data augmentation
    datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )
    
    model = models.Sequential([
        layers.Conv2D(64, (3, 3), activation='relu', input_shape=(150, 150, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(256, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
    return model

model_3 = vgg_3_blocks()
history_3 = model_2.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator,
    verbose=1
)

c:\Users\aashm\OneDrive\Desktop\ES335_gradient_thinkers\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 0.7773 - loss: 0.4638 - val_accuracy: 0.7805 - val_loss: 0.4351
Epoch 2/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.8008 - loss: 0.4118 - val_accuracy: 0.7561 - val_loss: 0.6602
Epoch 3/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.7788 - loss: 0.4813 - val_accuracy: 0.7805 - val_loss: 0.4438
Epoch 4/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.8084 - loss: 0.3977 - val_accuracy: 0.6585 - val_loss: 0.8052
Epoch 5/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.8022 - loss: 0.5561 - val_accuracy: 0.7805 - val_loss: 0.4632
Epoch 6/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.7808 - loss: 0.4838 - val_accuracy: 0.8049 - val_loss: 0.3647
Epoch 7/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.7397 - loss: 0.5114 - val_accuracy: 0.8537 - val_loss: 0.4136
Epoch 8/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.8409 - loss: 0.4382 - val_accuracy: 0.8049 - val_loss: 0.4316
Epoch 9/

In [20]:
results_3 = evaluate_model(model_3, history_3)
print(f"Model 3 Results: {results_3}")

Model 3 Results: {'training_loss': 0.40730559825897217, 'training_accuracy': 0.8414633870124817, 'test_accuracy': 0.5121951103210449, 'num_parameters': 4828481}


### Transfer Learning using VGG16/VGG19 with Fine-tuning All Layer

In [5]:
def transfer_learning_full_finetune():
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
    base_model.trainable = True  # Fine-tune all layers

    model = models.Sequential([
        base_model,
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    return model

model_4 = transfer_learning_full_finetune()
history_4 = model_4.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator,
    verbose=1
)

c:\Users\aashm\OneDrive\Desktop\ES335_gradient_thinkers\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.6144 - loss: 0.6711

c:\Users\aashm\OneDrive\Desktop\ES335_gradient_thinkers\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


6/6 ━━━━━━━━━━━━━━━━━━━━ 29s 4s/step - accuracy: 0.6146 - loss: 0.6697 - val_accuracy: 0.8537 - val_loss: 0.5690
Epoch 2/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 23s 4s/step - accuracy: 0.8046 - loss: 0.5731 - val_accuracy: 0.8780 - val_loss: 0.4601
Epoch 3/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 47s 4s/step - accuracy: 0.8632 - loss: 0.4710 - val_accuracy: 0.9024 - val_loss: 0.3485
Epoch 4/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 23s 4s/step - accuracy: 0.9449 - loss: 0.3892 - val_accuracy: 0.9024 - val_loss: 0.2557
Epoch 5/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 23s 4s/step - accuracy: 0.9593 - loss: 0.2850 - val_accuracy: 0.9268 - val_loss: 0.1867
Epoch 6/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 22s 3s/step - accuracy: 0.9369 - loss: 0.2137 - val_accuracy: 0.9512 - val_loss: 0.1436
Epoch 7/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 22s 3s/step - accuracy: 0.9381 - loss: 0.1737 - val_accuracy: 0.9512 - val_loss: 0.1180
Epoch 8/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 23s 4s/step - accuracy: 0.9330 - loss: 0.2021 - val_accuracy: 0.9756 - val_loss: 0.0888
Epoch 9/10
6/6 ━━━━

In [ ]:
results_4 = evaluate_model(model_4, history_4)
print(f"Model 4 Results: {results_4}")

Model 3 Results: {'training_loss': 0.1472366452217102, 'training_accuracy': 0.9390243887901306, 'test_accuracy': 0.9512194991111755, 'num_parameters': 15763521}


### Transfer Learning using VGG16/VGG19 with Fine-tuning Only Final MLP Layers

In [7]:
def transfer_learning_finetune_fc():
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
    base_model.trainable = False  # Freeze convolutional layers

    model = models.Sequential([
        base_model,
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    return model

model_5 = transfer_learning_finetune_fc()
history_5 = model_5.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator,
    verbose=1
)

Epoch 1/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 17s 3s/step - accuracy: 0.7060 - loss: 0.6418 - val_accuracy: 0.5610 - val_loss: 0.6374
Epoch 2/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - accuracy: 0.7406 - loss: 0.5974 - val_accuracy: 0.6341 - val_loss: 0.6054
Epoch 3/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 13s 3s/step - accuracy: 0.7029 - loss: 0.6033 - val_accuracy: 0.7317 - val_loss: 0.5712
Epoch 4/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - accuracy: 0.8221 - loss: 0.5591 - val_accuracy: 0.7561 - val_loss: 0.5389
Epoch 5/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - accuracy: 0.7600 - loss: 0.5633 - val_accuracy: 0.7805 - val_loss: 0.5121
Epoch 6/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - accuracy: 0.7795 - loss: 0.5407 - val_accuracy: 0.7805 - val_loss: 0.4856
Epoch 7/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - accuracy: 0.8130 - loss: 0.5298 - val_accuracy: 0.8293 - val_loss: 0.4610
Epoch 8/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 13s 3s/step - accuracy: 0.7786 - loss: 0.5151 - val_accuracy: 0.8537 - val_loss: 0.4365
Epoch 9/

In [9]:
results_5 = evaluate_model(model_5, history_5)
print(f"Model 5 Results: {results_5}")

Model 5 Results: {'training_loss': 0.4564601182937622, 'training_accuracy': 0.8719512224197388, 'test_accuracy': 0.9024389982223511, 'num_parameters': 15763521}
